In [ ]:
import time
import json
import math
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from py2neo import Graph, Node, Relationship
%matplotlib inline

In [ ]:
graph = Graph("bolt://neo4j-quanta:7687", auth=('neo4j','myneo'))

print("Connected to graph database with {:,} nodes and {:,} relationships!".format(
    graph.database.primitive_counts['NumberOfNodeIdsInUse'], 
    graph.database.primitive_counts['NumberOfRelationshipIdsInUse']))

In [ ]:
def query_to_df(query, graph):
    print("Starting query...", end=" ")
    query_start_time = time.time()
    df = graph.run(query).to_data_frame()
    print("Done ({:.2f} minutes).".format((time.time()-query_start_time)/60))
    return df

In [ ]:
## Histogram of community size

# Get number of Louvain iterations run
n_iters = query_to_df("MATCH (a:NatureAuthor) RETURN size(a.louvain) as n_iters LIMIT 1", graph)['n_iters'][0]

# Plot histogram at each iteration
for i in range(n_iters):
    query = """
    MATCH (a:NatureAuthor) 
    RETURN 
        a.louvain[{}] as community_id, 
        COUNT(a) as community_size""".format(i)
    df = query_to_df(query, graph)
    plt.figure()
    sns.distplot(df['community_size'])